# Choosing a location for a well

Glavrosgosneft oil company needs to decide where to drill a new well.

The steps for choosing a location are usually as follows:
- In the selected region, characteristics for wells are collected: the quality of oil and the volume of its reserves;
- Build a model to predict the volume of reserves in new wells;
- Select wells with the highest value estimates;
- Determine the region with the maximum total profit of the selected wells.

We have been provided with oil samples in three regions. The characteristics for each well in the region are already known. It is necessary to build a model to determine the region where mining will bring the greatest profit and analyze the possible profits and risks with *Bootstrap* technology.

# Data description
The exploration data of the three regions are in the files:
- /datasets/geo_data_0.csv.
- /datasets/geo_data_1.csv.
- /datasets/geo_data_2.csv.
- id — unique well ID;
- f0, f1, f2 — three signs of dots (it doesn't matter what they mean, but the signs themselves are significant);
- product — the volume of reserves in the well (thousand barrels).

Task conditions:

* Only linear regression is suitable for training the model (the rest are not predictable enough).

* During the exploration of the region, 500 points are explored, from which 200 best ones are selected for development using machine learning.

* The budget for the development of wells in the region is 10 billion rubles.

* At current prices, one barrel of raw materials brings 450 rubles of income. The income from each unit of the product is 450 thousand rubles, since the volume is indicated in thousands of barrels.

* After the risk assessment, you need to leave only those regions in which the probability of losses is less than 2.5%. Among them, the region with the highest average profit is chosen.

Synthetic data: details of contracts and characteristics of deposits are not disclosed.

## Loading and preparing data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df_1 = pd.read_csv('datasets/geo_data_0.csv')
df_2 = pd.read_csv('datasets/geo_data_1.csv')
df_3 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
display(df_1)
display(df_2)
display(df_3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [4]:
print(df_1.info())
print()
print(df_2.info())
print()
print(df_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

In [5]:
display(df_1.describe())
print()
display(df_2.describe())
print()
display(df_3.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


There are no missing values in the data

In [6]:
print(df_1.duplicated().sum())
print(df_2.duplicated().sum())
print(df_3.duplicated().sum())

0
0
0


There are no obvious duplicates in the data

In [7]:
df_1.drop('id', axis=1, inplace=True)
df_2.drop('id', axis=1, inplace=True)
df_3.drop('id', axis=1, inplace=True)

Unnecessary id columns have been removed

### Conclusion

The data has been verified and is ready for further processing

## Model training and testing

In [8]:
# fix random_state
random_state=12345

In [9]:
def model_calc(df):
    features = df.drop('product',axis=1)
    target = df['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features,target,test_size=0.25,
                                                                                  random_state=random_state)
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions = model.predict(features_valid)
    print('Average stock of actual raw materials: ',round(target_valid.mean(),3))
    print('Average stock of predicted raw materials: ',round(predictions.mean(),3))
    print('RMSE: ',round(mean_squared_error(target_valid,predictions)**0.5,3))
    return predictions, features_train, features_valid, target_train, target_valid

In [10]:
print('FIRST FIELD:')
predictions_1, features_train_1, features_valid_1, target_train_1, target_valid_1 = model_calc(df_1)
print()
print('SECOND FIELD:')
predictions_2, features_train_2, features_valid_2, target_train_2, target_valid_2 = model_calc(df_2)
print()
print('THIRD FIELD:')
predictions_3, features_train_3, features_valid_3, target_train_3, target_valid_3 = model_calc(df_3)

FIRST FIELD:
Average stock of actual raw materials:  92.079
Average stock of predicted raw materials:  92.593
RMSE:  37.579

SECOND FIELD:
Average stock of actual raw materials:  68.723
Average stock of predicted raw materials:  68.729
RMSE:  0.893

THIRD FIELD:
Average stock of actual raw materials:  94.884
Average stock of predicted raw materials:  94.965
RMSE:  40.03


### Conclusion:

based on the average predicted reserves and RMSE, the most accurate model is the model of the second deposit (the smallest RMSE), but in terms of the number of reserves this deposit is less.

## Preparation for profit calculation

In [11]:
ALL_MONEY = 10000000000
BAR_1000 = 450000
TARGET_LOSS = 0.025

WELLS = 200
ONE_WELL = ALL_MONEY/WELLS
print('Costs per well: ', ONE_WELL)
TARGET_BAR = ONE_WELL/450000
print('Required average production for payback of one well (thousand barrels): ', round(TARGET_BAR,3))

Costs per well:  50000000.0
Required average production for payback of one well (thousand barrels):  111.111


### Conclusion

- The average sufficient volume of raw materials for the break-even development of a new well is 111.11 thousand bar.
- The average reserves for each region are (thousand barrels):
  * 92.59 (first)
  * 68.73 (second)
  * 94.97 (third)
- As we see the average indicators do not correspond to the minimum payback, we conclude about that there are significant risks
- It is also worth noting that the second region is significantly lower in terms of average production, but this is only an average value.

## Calculation of profit and risks

In [12]:
def profit(predictions, target_valid):
    predictions = pd.Series(predictions)
    profit = predictions.sort_values(ascending=False).head(200).index
    profit = target_valid.reset_index(drop=True).iloc[profit].sum()*BAR_1000 - ALL_MONEY
    return round(profit,3)

In [13]:
print('FIRST FIELD:')
print('Total profit:',profit(predictions_1,target_valid_1))
print()
print('SECOND FIELD:')
print('Total profit:',profit(predictions_2,target_valid_2))
print()
print('THIRD FIELD:')
print('Total profit:',profit(predictions_3,target_valid_3))

FIRST FIELD:
Total profit: 3320826043.14

SECOND FIELD:
Total profit: 2415086696.682

THIRD FIELD:
Total profit: 2710349963.6


In [14]:
state = np.random.RandomState(12345)
def bootstrap(predictions,target_valid):    
    values = []
    for i in range(1000):
        predictions = pd.Series(predictions)
        subsample = predictions.sample(n=500, replace=True, random_state=state)
        values.append(profit(subsample,target_valid))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)

    print('Average profit:',round(sum(values)/len(values),3))
    print('Upper bound of the confidence interval:', round(higher,3))
    print('Lower bound of the confidence interval:', round(lower,3))
    print('Risk of losses:',round(len(values[values < 0])/len(values)*100,3))

In [15]:
print('FIRST FIELD:')
bootstrap(predictions_1,target_valid_1)
print()
print('SECOND FIELD:')
bootstrap(predictions_2,target_valid_2)
print()
print('THIRD FIELD:')
bootstrap(predictions_3,target_valid_3)

FIRST FIELD:
Average profit: 396164984.802
Upper bound of the confidence interval: 909766941.553
Lower bound of the confidence interval: -111215545.89
Risk of losses: 6.9

SECOND FIELD:
Average profit: 461155817.277
Upper bound of the confidence interval: 862952060.264
Lower bound of the confidence interval: 78050810.752
Risk of losses: 0.7

THIRD FIELD:
Average profit: 392950475.171
Upper bound of the confidence interval: 934562914.551
Lower bound of the confidence interval: -112227625.379
Risk of losses: 6.5


### Conclusion

Based on the analysis, the implementation of the Second deposit is proposed:

1) It was found that the smallest RMSE has a second deposit;

2) During Bootstrap, it was revealed that the region with the highest average profit is the Second field;

3) The lowest risk of losses is also in the Second deposit, the risk is quite insignificant (<1%) compared to the potential profit